In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.internals.construction import to_arrays
from prompt_toolkit.utils import to_int
from pulp import *

In [2]:
def find_loops(edges):
    """
    Find all closed loops starting and ending at node 0 in a directed graph.

    Args:
        edges: List of tuples representing directed edges (from_node, to_node)

    Returns:
        List of lists, where each sublist contains tuples forming a closed loop from 0 to 0
    """
    # Create adjacency list for faster lookup
    adj = {}
    for start, end in edges:
        if start not in adj:
            adj[start] = []
        adj[start].append(end)

    def find_paths(current, path, visited):
        """
        Recursive helper function to find all paths from current node back to 0.
        """
        if current == 'Depot' and len(path) > 0:  # Found a loop back to 0
            return [path[:]]

        if current not in adj:  # Dead end
            return []

        paths = []
        for next_node in adj[current]:
            if (current, next_node) not in visited:  # Avoid using same edge twice
                path.append((current, next_node))
                visited.add((current, next_node))
                paths.extend(find_paths(next_node, path, visited))
                path.pop()
                visited.remove((current, next_node))
        return paths

    # Start DFS from node 0
    all_loops = find_paths('Depot', [], set())

    # Filter out non-minimal loops (loops that contain other loops)
    minimal_loops = []
    for loop in all_loops:
        # Check if this loop contains any other loop
        is_minimal = True
        visited_nodes = {node for edge in loop for node in edge}
        for other_loop in all_loops:
            if loop != other_loop:
                other_nodes = {node for edge in other_loop for node in edge}
                if other_nodes.issubset(visited_nodes) and len(other_loop) < len(loop):
                    is_minimal = False
                    break
        if is_minimal:
            minimal_loops.append(loop)

    return minimal_loops

## Cost matrix

In [3]:
cost_matrix = pd.read_csv('costMatrixDistance.csv', index_col=0)

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(float)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(str)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(str)

## Shop demands

In [4]:
shop_demands = pd.read_csv('2015_shop_locations.csv').set_index('id')[['demand(kg)', 'stage']].astype(float)
shop_demands = shop_demands.loc[shop_demands['stage'] == 1, 'demand(kg)']
shops = shop_demands.index
nodes = shops.copy()
nodes = np.append('Depot', nodes)

## Vehicle routing problem

In [5]:
# Vehicle capacity
van_capacity = 300

In [6]:
# Creates a list of tuples containing all the possible routes for transport
A = [(i,j) for i in nodes for j in nodes if i!=j]

In [7]:
A

[('Depot', 'ChIJwTC7A55ZwokRPNX9g7ngbaI'),
 ('Depot', 'ChIJU1XImaNZwokRutunetC8XeE'),
 ('Depot', 'ChIJG7L-TLVbwokRT36uIrwz2Mo'),
 ('Depot', 'ChIJ5cPkuBtgwokRn55JgpGqjFA'),
 ('Depot', 'ChIJ24V7r_31wokR-S71l2zqrwc'),
 ('Depot', 'ChIJiUJ1DI5ZwokRWdK6SPg9BOY'),
 ('Depot', 'ChIJYTxm0Gf2wokR5V0iFDQ-2x0'),
 ('Depot', 'ChIJ_eQYpFdmwokR756MAH2tZPw'),
 ('Depot', 'ChIJYzaRC2REwokRaH2rHpflSYk'),
 ('Depot', 'ChIJ3R85DSb2wokR1XobETRMs5E'),
 ('Depot', 'ChIJ97BgiDtfwokRsbkP6Vl1seY'),
 ('Depot', 'ChIJT1ZEl4JYwokRXQMvGg9VMPE'),
 ('Depot', 'ChIJ7-uIlEFEwokRlbHGtjrdFtM'),
 ('Depot', 'ChIJU3IVO-tZwokRx6iWrhKQTng'),
 ('Depot', 'ChIJRR7W62BFwokRlrVks40mAdU'),
 ('Depot', 'ChIJnZ4lShdkwokRQUAn5mjU1pA'),
 ('Depot', 'ChIJG9cUYg_1wokRUNpvwM84Z4M'),
 ('Depot', 'ChIJmZQsMgFbwokRMc5I7kPdQ7w'),
 ('Depot', 'ChIJt5DcjLRFwokRyxil5Pjp3Fo'),
 ('Depot', 'ChIJ2b9se6f1wokRUN8kWpFfzN0'),
 ('ChIJwTC7A55ZwokRPNX9g7ngbaI', 'Depot'),
 ('ChIJwTC7A55ZwokRPNX9g7ngbaI', 'ChIJU1XImaNZwokRutunetC8XeE'),
 ('ChIJwTC7A55ZwokRPNX9g7ngbaI',

In [8]:
# A set of variables x is created to contain the vehicle routes variables (binary var)
x = {}
for a in A:
    x[a] = LpVariable("x(%s,%s)" %a, cat=LpBinary)

# A set of variables u is created to contain the continuous delivered quantity
u = {}
for shop in shops:
    u[shop] = LpVariable("u(%s)" %shop, shop_demands[shop], van_capacity)

In [9]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("CVRP_Problem",LpMinimize)

In [10]:
# The objective function is added to 'prob' first
prob += lpSum(cost_matrix.loc[i,j]*x[i,j] for (i,j) in A)

In [11]:
#Constraint 1 : imposes that exactly one arc enters each customer node
one_visit_out = {}
for shop in shops:
    one_visit_out[shop] = lpSum([x[shop,node] for node in nodes if shop!=node]) == 1.0
    prob += one_visit_out[shop]

In [12]:
#Constraint 2 : imposes that exactly one arc leaves each customer node
one_visit_in = {}
for shop in shops:
    one_visit_in[shop] = lpSum([x[node,shop] for node in nodes if node!=shop]) == 1.0
    prob += one_visit_in[shop]

In [13]:
#Constraint 4 : imposes the amount of flow that leaves the depot should be identical
#with the flow returns to the depot
depot_node = lpSum([x['Depot',shop] for shop in shops]) == lpSum([x[shop,'Depot'] for shop in shops])
prob += depot_node

In [14]:
#Constraint 5 : vehicle capacity constraint and sub tour elimination constraint
sub_tours_cap = {}
for shop_out in shops:
    for shop_in in shops:
        if shop_out != shop_in:
            sub_tours_cap[shop_out,shop_in] = u[shop_out] - u[shop_in] + van_capacity * x[shop_out,shop_in] <= van_capacity - shop_demands[shop_in]
            prob += sub_tours_cap[shop_out,shop_in]

In [21]:
# The problem is solved using PuLP's choice of Solver (ici Cplex, retourne 1 si résolu, -1 sinon)
prob.solve(GUROBI(msg=False, timeLimit=600))

1

In [22]:
# The status of the solution is printed to the screen (optimal ou pas)
print("Status:", LpStatus[prob.status])

Status: Optimal


In [23]:
# Each of the variables is printed with it's resolved optimum value
for key,val in x.items():
    if val.varValue==1.0:
        print(val,"=",val.varValue)

x(Depot,ChIJ97BgiDtfwokRsbkP6Vl1seY) = 1.0
x(Depot,ChIJmZQsMgFbwokRMc5I7kPdQ7w) = 1.0
x(ChIJwTC7A55ZwokRPNX9g7ngbaI,Depot) = 1.0
x(ChIJU1XImaNZwokRutunetC8XeE,ChIJwTC7A55ZwokRPNX9g7ngbaI) = 1.0
x(ChIJG7L_TLVbwokRT36uIrwz2Mo,ChIJRR7W62BFwokRlrVks40mAdU) = 1.0
x(ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ_eQYpFdmwokR756MAH2tZPw) = 1.0
x(ChIJ24V7r_31wokR_S71l2zqrwc,ChIJT1ZEl4JYwokRXQMvGg9VMPE) = 1.0
x(ChIJiUJ1DI5ZwokRWdK6SPg9BOY,Depot) = 1.0
x(ChIJYTxm0Gf2wokR5V0iFDQ_2x0,ChIJ2b9se6f1wokRUN8kWpFfzN0) = 1.0
x(ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJnZ4lShdkwokRQUAn5mjU1pA) = 1.0
x(ChIJYzaRC2REwokRaH2rHpflSYk,ChIJ7_uIlEFEwokRlbHGtjrdFtM) = 1.0
x(ChIJ3R85DSb2wokR1XobETRMs5E,ChIJYTxm0Gf2wokR5V0iFDQ_2x0) = 1.0
x(ChIJ97BgiDtfwokRsbkP6Vl1seY,ChIJ24V7r_31wokR_S71l2zqrwc) = 1.0
x(ChIJT1ZEl4JYwokRXQMvGg9VMPE,ChIJ3R85DSb2wokR1XobETRMs5E) = 1.0
x(ChIJ7_uIlEFEwokRlbHGtjrdFtM,ChIJt5DcjLRFwokRyxil5Pjp3Fo) = 1.0
x(ChIJU3IVO_tZwokRx6iWrhKQTng,ChIJiUJ1DI5ZwokRWdK6SPg9BOY) = 1.0
x(ChIJRR7W62BFwokRlrVks40mAdU,ChIJYzaRC2REwokRaH

In [24]:
# The optimised objective function value is printed to the screen
print("Total Cost = ", value(prob.objective))

Total Cost =  73.66304926808475


In [25]:
# Afficher les acrs correspondant à la solution
active_arcs = [key for key,val in x.items() if val.varValue >=0.9 ]
find_loops(active_arcs)


[[('Depot', 'ChIJ97BgiDtfwokRsbkP6Vl1seY'),
  ('ChIJ97BgiDtfwokRsbkP6Vl1seY', 'ChIJ24V7r_31wokR-S71l2zqrwc'),
  ('ChIJ24V7r_31wokR-S71l2zqrwc', 'ChIJT1ZEl4JYwokRXQMvGg9VMPE'),
  ('ChIJT1ZEl4JYwokRXQMvGg9VMPE', 'ChIJ3R85DSb2wokR1XobETRMs5E'),
  ('ChIJ3R85DSb2wokR1XobETRMs5E', 'ChIJYTxm0Gf2wokR5V0iFDQ-2x0'),
  ('ChIJYTxm0Gf2wokR5V0iFDQ-2x0', 'ChIJ2b9se6f1wokRUN8kWpFfzN0'),
  ('ChIJ2b9se6f1wokRUN8kWpFfzN0', 'ChIJG9cUYg_1wokRUNpvwM84Z4M'),
  ('ChIJG9cUYg_1wokRUNpvwM84Z4M', 'ChIJ5cPkuBtgwokRn55JgpGqjFA'),
  ('ChIJ5cPkuBtgwokRn55JgpGqjFA', 'ChIJ_eQYpFdmwokR756MAH2tZPw'),
  ('ChIJ_eQYpFdmwokR756MAH2tZPw', 'ChIJnZ4lShdkwokRQUAn5mjU1pA'),
  ('ChIJnZ4lShdkwokRQUAn5mjU1pA', 'ChIJU3IVO-tZwokRx6iWrhKQTng'),
  ('ChIJU3IVO-tZwokRx6iWrhKQTng', 'ChIJiUJ1DI5ZwokRWdK6SPg9BOY'),
  ('ChIJiUJ1DI5ZwokRWdK6SPg9BOY', 'Depot')],
 [('Depot', 'ChIJmZQsMgFbwokRMc5I7kPdQ7w'),
  ('ChIJmZQsMgFbwokRMc5I7kPdQ7w', 'ChIJG7L-TLVbwokRT36uIrwz2Mo'),
  ('ChIJG7L-TLVbwokRT36uIrwz2Mo', 'ChIJRR7W62BFwokRlrVks40mAdU'),
  ('ChIJR

In [26]:
# Each of the variables is printed with it's resolved optimum value
for key,val in u.items():
    print(val,"=",val.varValue)

u(ChIJwTC7A55ZwokRPNX9g7ngbaI) = 300.0
u(ChIJU1XImaNZwokRutunetC8XeE) = 279.0
u(ChIJG7L_TLVbwokRT36uIrwz2Mo) = 44.0
u(ChIJ5cPkuBtgwokRn55JgpGqjFA) = 194.0
u(ChIJ24V7r_31wokR_S71l2zqrwc) = 36.0
u(ChIJiUJ1DI5ZwokRWdK6SPg9BOY) = 300.0
u(ChIJYTxm0Gf2wokR5V0iFDQ_2x0) = 105.0
u(ChIJ_eQYpFdmwokR756MAH2tZPw) = 228.0
u(ChIJYzaRC2REwokRaH2rHpflSYk) = 83.0
u(ChIJ3R85DSb2wokR1XobETRMs5E) = 87.0
u(ChIJ97BgiDtfwokRsbkP6Vl1seY) = 19.0
u(ChIJT1ZEl4JYwokRXQMvGg9VMPE) = 62.0
u(ChIJ7_uIlEFEwokRlbHGtjrdFtM) = 225.0
u(ChIJU3IVO_tZwokRx6iWrhKQTng) = 285.0
u(ChIJRR7W62BFwokRlrVks40mAdU) = 67.0
u(ChIJnZ4lShdkwokRQUAn5mjU1pA) = 261.0
u(ChIJG9cUYg_1wokRUNpvwM84Z4M) = 164.0
u(ChIJmZQsMgFbwokRMc5I7kPdQ7w) = 23.0
u(ChIJt5DcjLRFwokRyxil5Pjp3Fo) = 246.0
u(ChIJ2b9se6f1wokRUN8kWpFfzN0) = 133.0
